In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, Conv1D
from keras.layers import LSTM 
from keras.layers import Reshape
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from os import listdir
import cv2

Using TensorFlow backend.


In [2]:
imgs = []
greys = []
for i in listdir('pics'):
    imgs.append(cv2.imread('pics/%s'%i, cv2.COLOR_BGR2RGB))

In [3]:
for i,e in enumerate(imgs):
    imgs[i] = cv2.resize(e, (128,128))
    greys.append(cv2.cvtColor(imgs[i], cv2.COLOR_RGB2GRAY))

In [4]:
# for lstm model
enco_dict = {}
deco_dict = {}

In [5]:
def pad_digit(digit):
    if digit < 10:
        _digit = "00" + str(digit)
    elif digit < 100:
        _digit = "0" + str(digit)
    else:
        _digit = str(digit)
    return _digit

In [6]:
encoder_list = []
imgs_encoded = []
for i in imgs:
    _img = []
    for j in i:
        for k in j:
            _ = pad_digit(k[0]) + pad_digit(k[1]) + pad_digit(k[2])
            encoder_list.append(_)
            _img.append(_)
    imgs_encoded.append(_img)

In [7]:
enco_dict = {e:i for i,e in enumerate(np.unique(encoder_list))}
deco_dict = {v: k for k, v in enco_dict.items()}

In [8]:
lstm_imgs = []
for i in imgs_encoded:
    _img = []
    for j in i:
        _img.append(enco_dict[j])
    lstm_imgs.append(_img)

In [10]:
# def define_model():
#     model = Sequential()
#     model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1, 128, 128)))
# #     model.add(MaxPooling2D((2, 2)))
#     model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
#     model.add(Conv2D(3, (3, 3), activation='relu', kernel_initializer='he_uniform'))
# #     # compile model
#     opt = SGD(lr=0.01, momentum=0.9)
#     model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
#     return model

In [11]:
# evaluate a model using k-fold cross-validation
def evaluate_model(dataX, dataY, n_folds=5):
    scores, histories = list(), list()
    # prepare cross validation
    kfold = KFold(n_folds, shuffle=True, random_state=1)
    # enumerate splits
    for train_ix, test_ix in kfold.split(dataX):
        # define model
        model = define_model()
        # select rows for train and test
        trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
        # fit model
        history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
        # evaluate model
        _, acc = model.evaluate(testX, testY, verbose=0)
        print('> %.3f' % (acc * 100.0))
        # stores scores
        scores.append(acc)
        histories.append(history)
        return scores, histories

In [12]:
# plot diagnostic learning curves
def summarize_diagnostics(histories):
    for i in range(len(histories)):
        # plot loss
        pyplot.subplot(2, 1, 1)
        pyplot.title('Cross Entropy Loss')
        pyplot.plot(histories[i].history['loss'], color='blue', label='train')
        pyplot.plot(histories[i].history['val_loss'], color='orange', label='test')
        # plot accuracy
        pyplot.subplot(2, 1, 2)
        pyplot.title('Classification Accuracy')
        pyplot.plot(histories[i].history['accuracy'], color='blue', label='train')
        pyplot.plot(histories[i].history['val_accuracy'], color='orange', label='test')
    pyplot.show()

In [13]:
# summarize model performance
def summarize_performance(scores):
    # print summary
    print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))
    # box and whisker plots of results
    pyplot.boxplot(scores)
    pyplot.show()

In [14]:
# run the test harness for evaluating a model
def run_test_harness():
    # load dataset
    trainX, trainY, testX, testY = greys, lstm_imgs, greys, lstm_imgs
    # prepare pixel data
#     trainX, testX = prep_pixels(trainX, testX)
    # evaluate model
    
#     scores, histories = evaluate_model(trainX, trainY)
#     # learning curves
#     summarize_diagnostics(histories)
#     # summarize estimated performance
#     summarize_performance(scores)

In [15]:
# entry point, run the test harness
run_test_harness()

In [32]:
# define cnn model
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu',data_format="channels_last", kernel_initializer='he_uniform', input_shape=(128, 128, 1)))
#     model.add(MaxPooling2D((2, 2)))
#     model.add(Flatten((-1,1,1)))
    model.add(Reshape((-1)))
#     model.add(Conv1D(32,3, activation='relu', kernel_initializer='he_uniform'))
    model.add(LSTM(256, activation='relu', kernel_initializer='he_uniform'))
    model.add(LSTM(512, activation='relu', kernel_initializer='he_uniform'))
    model.add(LSTM(1024, activation='relu', kernel_initializer='he_uniform'))
    model.add(LSTM(2048, activation='relu', kernel_initializer='he_uniform'))
    model.add(LSTM(8192, activation='relu', kernel_initializer='he_uniform'))
    model.add(LSTM(16384, activation='relu', kernel_initializer='he_uniform'))
    # compile model
    opt = SGD(lr=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [33]:
m = define_model()

ValueError: Input 0 is incompatible with layer lstm_8: expected ndim=3, found ndim=2

In [31]:
126*126*32/128

3969.0